# DLC-style visualization of tracked points

## First: what format does DLC expect?

In [1]:
import os
import deeplabcut
from DLC_for_WBFM.bin.configuration_definition import *
import pandas as pd

In [2]:
# Get the format for a DLC annotation
proj_folder = r'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_Projects\wbfm_3d-Charlie-2020-11-02'
annotations = r'videos\test2020-10-22_mcherry_fr1000_sl10_15DLC_resnet50_wbfm_3dNov2shuffle1_200000.h5'

fname = os.path.join(proj_folder, annotations)


In [3]:
dlc_df = pd.read_hdf(fname)

In [4]:
# list(dlc_df.columns)

In [5]:
dlc_df

scorer    DLC_resnet50_wbfm_3dNov2shuffle1_200000                         \
bodyparts                                 Unknown                          
coords                                          x           y likelihood   
0                                      383.242126  339.124512   0.999886   
1                                      349.152527  331.157623   0.999123   
2                                      318.318909  300.538666   0.999827   
3                                      330.236450  249.308975   0.999657   
4                                      350.949280  245.348846   0.999941   
...                                           ...         ...        ...   
996                                    434.268951  216.251358   0.999906   
997                                    408.076385  210.307434   0.999913   
998                                    391.237793  220.066818   0.999219   
999                                    369.182953  240.666199   0.999690   
1000                                   355.310272  254.927307   0.999868   

scorer                                                                \
bodyparts        REV1                               REV2               
coords              x           y likelihood           x           y   
0          334.708984  255.272385   0.995434  332.930573  246.088257   
1          334.132416  237.868362   0.996484  337.838257  227.709763   
2          351.440125  210.012589   0.998936  361.244141  204.466827   
3          408.621246  194.238434   0.999533  418.854095  194.650040   
4          433.580170  199.993912   0.999419  444.895874  197.019913   
...               ...         ...        ...         ...         ...   
996        442.489319  311.700165   0.998914  439.349731  322.669098   
997        442.697815  298.347015   0.998234  444.694855  308.898132   
998        453.541656  293.015839   0.999162  460.126190  301.793701   
999        458.634033  286.251465   0.998050  469.602753  289.305664   
1000       458.274475  272.144531   0.996989  467.893951  272.880646   

scorer                            ...                                     \
bodyparts                   REV3  ...      SMDD1       SMDD2               
coords    likelihood           x  ... likelihood           x           y   
0           0.997458  321.893524  ...   0.998590  402.246552  248.830399   
1           0.998751  327.781067  ...   0.999349  398.665771  254.686859   
2           0.997530  358.966797  ...   0.999717  397.303589  258.664001   
3           0.999379  419.176727  ...   0.999893  420.335632  266.022278   
4           0.998115  444.856262  ...   0.999918  441.601288  268.324615   
...              ...         ...  ...        ...         ...         ...   
996         0.996215  451.450226  ...   0.994672  376.268188  279.032532   
997         0.993779  457.173431  ...   0.990499  375.010162  289.911743   
998         0.994579  471.689514  ...   0.995730  388.532837  308.780426   
999         0.997274  478.595734  ...   0.999649  402.801147  325.277191   
1000        0.998118  476.843201  ...   0.999521  412.535706  323.896851   

scorer                                                               \
bodyparts                   URYV                               URYD   
coords    likelihood           x           y likelihood           x   
0           0.998011  304.352844  186.054459   0.991996  276.372711   
1           0.999662  335.647583  156.690689   0.992330  308.421539   
2           0.999139  397.077362  138.286743   0.996410  374.397583   
3           0.999517  474.055969  143.743805   0.992864  447.357849   
4           0.999411  493.701538  139.091354   0.996947  463.264771   
...              ...         ...         ...        ...         ...   
996         0.980528  477.430511  392.496613   0.996866  495.529785   
997         0.991555  509.703278  361.363922   0.983882  514.036743   
998         0.996222  539.220520  326.578522   0.985352  530.419800   
999      

## Import tracklets and write in this format

In [6]:
import numpy as np
from tqdm import tqdm

In [76]:
project_folder = r'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment'
fname = os.path.join(project_folder, 'clust_df_dat.pickle')
clust_df = pd.read_pickle(fname)

In [77]:
# Writing options
is_3d = False


In [84]:
def build_dlc_annotation_one_tracklet(row,
                                      all_bodyparts,
                                      num_frames=1000,
                                      coord_names=['x','y','likelihood'],
                                      min_length=5,
                                      neuron_ind=1,
                                      verbose=0):
    # TODO: Check z
    
    # Variables to be written
    scorer = 'feature_tracker'
    all_frames = list(range(num_frames))
    
    tracklet_length = len(row['all_xyz'])
    if tracklet_length < min_length:
        return None
    
    if verbose >= 1:
        print(f"Found tracklet of length {tracklet_length}")
    
    # Build a dataframe for one neuron across all frames
    # Will be zeros if not detected in a given frame
    coords = np.zeros((num_frames,len(coord_names),))
    for this_slice, this_xyz, this_prob in zip(row['slice_ind'], row['all_xyz'], row['all_prob']):
        # TODO: only works for xy; this_xyz is format ZXY
        coords[this_slice,0] = this_xyz[1]
        coords[this_slice,1] = this_xyz[2]
        try:
            coords[this_slice,-1] = this_prob
        except:
            coords[this_slice,-1] = 0.0
            pass

    index = pd.MultiIndex.from_product([[scorer], [f'neuron{neuron_ind}'],
                                        coord_names],
                                        names=['scorer', 'bodyparts', 'coords'])
    frame = pd.DataFrame(coords, columns = index, index = all_frames)

    return frame


def build_dlc_annotation_all(clust_df, min_length, verbose=0):
    new_dlc_df = None
    all_bodyparts = np.asarray(clust_df['clust_ind'])
    
    neuron_ind = 1
    for i, row in tqdm(clust_df.iterrows(), total=clust_df.shape[0]):
        opt = {'min_length':min_length, 'verbose':verbose-1,'neuron_ind':neuron_ind}
        frame = build_dlc_annotation_one_tracklet(row, all_bodyparts, **opt)
        if frame is not None:
            new_dlc_df = pd.concat([new_dlc_df, frame],axis=1)
            neuron_ind = neuron_ind + 1
        
#         if verbose >= 1:
#             print("============================")
#             print(f"Row {i}/{len(all_bodyparts)}")
    
    return new_dlc_df
        

In [85]:
new_dlc_df = build_dlc_annotation_all(clust_df, min_length=50, verbose=0)

100%|███████████████████████████████████████████████████| 13431/13431 [00:01<00:00, 7077.22it/s]


In [86]:
scorer = 'feature_tracker'

# output_path = os.path.join(project_folder)
new_dlc_df.to_csv(os.path.join(project_folder,"CollectedData_" + scorer + ".csv"))
new_dlc_df.to_hdf(os.path.join(project_folder,"CollectedData_" + scorer + ".h5"), key="df_with_missing", mode="w")
new_dlc_df.to_pickle(os.path.join(project_folder,"CollectedData_" + scorer + ".pickle"))
# deeplabcut.convertcsv2h5(path_config_file, userfeedback=False)

In [68]:
df

,clust_ind,all_ind_local,all_ind_global,all_xyz,all_prob,slice_ind,extended_this_slice,not_finished
0,0,"[5, 6]","[5, 148]","[[7.5, 513.4677276611328, 98.7389030456543], [...",[0.3333333333333333],"[0, 1]",False,False
1,1,"[6, 2, 11, 12, 12, 23, 5, 7, 6, 10, 5, 7, 54, ...","[6, 144, 293, 436, 575, 725, 850, 989, 1119, 1...","[[10.0, 508.24971424449575, 103.97156108509411...","[0.5581395348837209, 0.6857142857142857, 0.687...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False
2,2,"[8, 35, 37]","[8, 177, 319]","[[9.0, 384.00681559244794, 229.6750233968099],...","[0.375, 0.4406779661016949]","[0, 1, 2]",False,False
3,3,"[11, 15, 3, 21, 11, 3, 13, 18, 23, 11, 12, 11,...","[11, 157, 285, 445, 574, 705, 858, 1000, 1136,...","[[9.5, 411.021183013916, 311.0642318725586], [...","[0.56, 0.6571428571428571, 0.2727272727272727,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False
4,4,"[12, 12, 2, 1, 9, 17, 28, 22, 18, 12, 8, 17, 15]","[12, 154, 284, 425, 572, 719, 873, 1004, 1131,...","[[8.5, 418.8942362467448, 267.5903727213542], ...","[0.3333333333333333, 0.6896551724137931, 0.608...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",False,False
...,...,...,...,...,...,...,...,...
13426,13426,"[69, 69]","[136041, 136168]","[[19.0, 404.9707530628551, 354.6851834383878],...",[0.6153846153846154],"[998, 999]",True,True
13427,13427,"[74, 83]","[136046, 136182]","[[18.5, 442.26753997802734, 153.6902256011963]...",[0.4444444444444444],"[998, 999]",True,True
13428,13428,"[89, 110]","[136061, 136209]","[[22.0, 426.085700141059, 255.0216996934679], ...",[0.2962962962962963],"[998, 999]",True,True
13429,13429,"[96, 112]","[136068, 136211]","[[22.0, 392.5264456612723, 382.9610290527344],...",[0.35714285714285715],"[998, 999]",True,True


In [87]:
new_dlc_df

scorer    feature_tracker                                                 \
bodyparts         neuron1                        neuron2                   
coords                  x           y likelihood       x    y likelihood   
0              405.615550  176.577909   0.631579     0.0  0.0        0.0   
1              422.131641  192.162372   0.728814     0.0  0.0        0.0   
2              415.586197  192.361758   0.760000     0.0  0.0        0.0   
3              379.441018  178.063309   0.812500     0.0  0.0        0.0   
4              388.632634  174.589680   0.847458     0.0  0.0        0.0   
..                    ...         ...        ...     ...  ...        ...   
995              0.000000    0.000000   0.000000     0.0  0.0        0.0   
996              0.000000    0.000000   0.000000     0.0  0.0        0.0   
997              0.000000    0.000000   0.000000     0.0  0.0        0.0   
998              0.000000    0.000000   0.000000     0.0  0.0        0.0   
999              0.000000    0.000000   0.000000     0.0  0.0        0.0   

scorer                                     ...                            \
bodyparts neuron3                 neuron4  ...  neuron113 neuron114        
coords          x    y likelihood       x  ... likelihood         x    y   
0             0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
1             0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
2             0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
3             0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
4             0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
..            ...  ...        ...     ...  ...        ...       ...  ...   
995           0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
996           0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
997           0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
998           0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   
999           0.0  0.0        0.0     0.0  ...        0.0       0.0  0.0   

scorer                                                               \
bodyparts              neuron115                          neuron116   
coords    likelihood           x           y likelihood           x   
0                0.0    0.000000    0.000000   0.000000    0.000000   
1                0.0    0.000000    0.000000   0.000000    0.000000   
2                0.0    0.000000    0.000000   0.000000    0.000000   
3                0.0    0.000000    0.000000   0.000000    0.000000   
4                0.0    0.000000    0.000000   0.000000    0.000000   
..               ...         ...         ...        ...         ...   
995              0.0  454.297062  266.601676   0.847458  459.583107   
996              0.0  449.973145  265.449445   0.779661  453.057814   
997              0.0  438.309376  281.978748   0.830508  433.233490   
998              0.0  429.115683  284.743779   0.620000  418.721970   
999              0.0    0.000000    0.000000   0.000000    0.000000   

scorer                            
bodyparts                         
coords              y likelihood  
0            0.000000   0.000000  
1            0.000000   0.000000  
2            0.000000   0.000000  
3            0.000000   0.000000  
4            0.000000   0.000000  
..                ...        ...  
995        318.398365   0.640000  
996        315.003845   0.830508  
997        326.272789   0.740000  
998        323.400276   0.847458  
999          0.000000   0.000000  

[1000 rows x 348 columns]

# Make a raw and labeled .avi video

## Use a config file to organize everything

In [14]:
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.bin.configuration_definition import *
import os

In [15]:
# Define the project folder

# Initialize the config files
task_name = 'feature_alignment'
experimenter = 'Charlie'

config = DLCForWBFMConfig(task_name=task_name, experimenter=experimenter)

# Note: Windows-style filename
working_directory = '.'

config = create_project(config, working_directory)

Project "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-01-06" already exists!


In [16]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

red_avi_fname = None
green_avi_fname = None

# Create the first sub-object
# This remembers the location of the raw data
datafiles = DLCForWBFMDatafiles(btf_fname_red, btf_fname_green, red_avi_fname, green_avi_fname)

config.datafiles = datafiles
save_config(config)

In [17]:
# Set up options for the .avi video
opt = {'start_volume':           15, 
       'num_frames':             1000, 
       'num_total_slices':       33, 
       'num_crop_slices':        11, 
       'center_slice':           12, 
       'alpha':                  0.15,
       'fps':                    5.0,
       'red_and_green_mirrored': True}
preprocessing = DLCForWBFMPreprocessing(**opt)

config.preprocessing = preprocessing
save_config(config)

In [36]:
# Actually write the video
from DLC_for_WBFM.utils.video_and_data_conversion.video_conversion_utils import write_minimax_projection_from_btf

write_minimax_projection_from_btf(config)

Converting volume index 15 to frame index 495
Taking a max of 11 slices, starting at 7
Page 502/33000; a portion of slice 0/1000 to tmp array index 0
Page 503/33000; a portion of slice 0/1000 to tmp array index 1
Page 504/33000; a portion of slice 0/1000 to tmp array index 2
Page 505/33000; a portion of slice 0/1000 to tmp array index 3
Page 506/33000; a portion of slice 0/1000 to tmp array index 4
Page 507/33000; a portion of slice 0/1000 to tmp array index 5
Page 508/33000; a portion of slice 0/1000 to tmp array index 6
Page 509/33000; a portion of slice 0/1000 to tmp array index 7
Page 510/33000; a portion of slice 0/1000 to tmp array index 8
Page 511/33000; a portion of slice 0/1000 to tmp array index 9
Page 512/33000; a portion of slice 0/1000 to tmp array index 10
Page 535/33000; a portion of slice 1/1000 to tmp array index 0
Page 536/33000; a portion of slice 1/1000 to tmp array index 1
Page 537/33000; a portion of slice 1/1000 to tmp array index 2
Page 538/33000; a portion of s

Page 931/33000; a portion of slice 13/1000 to tmp array index 0
Page 932/33000; a portion of slice 13/1000 to tmp array index 1
Page 933/33000; a portion of slice 13/1000 to tmp array index 2
Page 934/33000; a portion of slice 13/1000 to tmp array index 3
Page 935/33000; a portion of slice 13/1000 to tmp array index 4
Page 936/33000; a portion of slice 13/1000 to tmp array index 5
Page 937/33000; a portion of slice 13/1000 to tmp array index 6
Page 938/33000; a portion of slice 13/1000 to tmp array index 7
Page 939/33000; a portion of slice 13/1000 to tmp array index 8
Page 940/33000; a portion of slice 13/1000 to tmp array index 9
Page 941/33000; a portion of slice 13/1000 to tmp array index 10
Page 964/33000; a portion of slice 14/1000 to tmp array index 0
Page 965/33000; a portion of slice 14/1000 to tmp array index 1
Page 966/33000; a portion of slice 14/1000 to tmp array index 2
Page 967/33000; a portion of slice 14/1000 to tmp array index 3
Page 968/33000; a portion of slice 14/1

Page 1327/33000; a portion of slice 25/1000 to tmp array index 0
Page 1328/33000; a portion of slice 25/1000 to tmp array index 1
Page 1329/33000; a portion of slice 25/1000 to tmp array index 2
Page 1330/33000; a portion of slice 25/1000 to tmp array index 3
Page 1331/33000; a portion of slice 25/1000 to tmp array index 4
Page 1332/33000; a portion of slice 25/1000 to tmp array index 5
Page 1333/33000; a portion of slice 25/1000 to tmp array index 6
Page 1334/33000; a portion of slice 25/1000 to tmp array index 7
Page 1335/33000; a portion of slice 25/1000 to tmp array index 8
Page 1336/33000; a portion of slice 25/1000 to tmp array index 9
Page 1337/33000; a portion of slice 25/1000 to tmp array index 10
Page 1360/33000; a portion of slice 26/1000 to tmp array index 0
Page 1361/33000; a portion of slice 26/1000 to tmp array index 1
Page 1362/33000; a portion of slice 26/1000 to tmp array index 2
Page 1363/33000; a portion of slice 26/1000 to tmp array index 3
Page 1364/33000; a porti

Page 1723/33000; a portion of slice 37/1000 to tmp array index 0
Page 1724/33000; a portion of slice 37/1000 to tmp array index 1
Page 1725/33000; a portion of slice 37/1000 to tmp array index 2
Page 1726/33000; a portion of slice 37/1000 to tmp array index 3
Page 1727/33000; a portion of slice 37/1000 to tmp array index 4
Page 1728/33000; a portion of slice 37/1000 to tmp array index 5
Page 1729/33000; a portion of slice 37/1000 to tmp array index 6
Page 1730/33000; a portion of slice 37/1000 to tmp array index 7
Page 1731/33000; a portion of slice 37/1000 to tmp array index 8
Page 1732/33000; a portion of slice 37/1000 to tmp array index 9
Page 1733/33000; a portion of slice 37/1000 to tmp array index 10
Page 1756/33000; a portion of slice 38/1000 to tmp array index 0
Page 1757/33000; a portion of slice 38/1000 to tmp array index 1
Page 1758/33000; a portion of slice 38/1000 to tmp array index 2
Page 1759/33000; a portion of slice 38/1000 to tmp array index 3
Page 1760/33000; a porti

Page 2119/33000; a portion of slice 49/1000 to tmp array index 0
Page 2120/33000; a portion of slice 49/1000 to tmp array index 1
Page 2121/33000; a portion of slice 49/1000 to tmp array index 2
Page 2122/33000; a portion of slice 49/1000 to tmp array index 3
Page 2123/33000; a portion of slice 49/1000 to tmp array index 4
Page 2124/33000; a portion of slice 49/1000 to tmp array index 5
Page 2125/33000; a portion of slice 49/1000 to tmp array index 6
Page 2126/33000; a portion of slice 49/1000 to tmp array index 7
Page 2127/33000; a portion of slice 49/1000 to tmp array index 8
Page 2128/33000; a portion of slice 49/1000 to tmp array index 9
Page 2129/33000; a portion of slice 49/1000 to tmp array index 10
Page 2152/33000; a portion of slice 50/1000 to tmp array index 0
Page 2153/33000; a portion of slice 50/1000 to tmp array index 1
Page 2154/33000; a portion of slice 50/1000 to tmp array index 2
Page 2155/33000; a portion of slice 50/1000 to tmp array index 3
Page 2156/33000; a porti

...

Page 32149/33000; a portion of slice 959/1000 to tmp array index 0
Page 32150/33000; a portion of slice 959/1000 to tmp array index 1
Page 32151/33000; a portion of slice 959/1000 to tmp array index 2
Page 32152/33000; a portion of slice 959/1000 to tmp array index 3
Page 32153/33000; a portion of slice 959/1000 to tmp array index 4
Page 32154/33000; a portion of slice 959/1000 to tmp array index 5
Page 32155/33000; a portion of slice 959/1000 to tmp array index 6
Page 32156/33000; a portion of slice 959/1000 to tmp array index 7
Page 32157/33000; a portion of slice 959/1000 to tmp array index 8
Page 32158/33000; a portion of slice 959/1000 to tmp array index 9
Page 32159/33000; a portion of slice 959/1000 to tmp array index 10
Page 32182/33000; a portion of slice 960/1000 to tmp array index 0
Page 32183/33000; a portion of slice 960/1000 to tmp array index 1
Page 32184/33000; a portion of slice 960/1000 to tmp array index 2
Page 32185/33000; a portion of slice 960/1000 to tmp array in

Page 32555/33000; a portion of slice 971/1000 to tmp array index 10
Page 32578/33000; a portion of slice 972/1000 to tmp array index 0
Page 32579/33000; a portion of slice 972/1000 to tmp array index 1
Page 32580/33000; a portion of slice 972/1000 to tmp array index 2
Page 32581/33000; a portion of slice 972/1000 to tmp array index 3
Page 32582/33000; a portion of slice 972/1000 to tmp array index 4
Page 32583/33000; a portion of slice 972/1000 to tmp array index 5
Page 32584/33000; a portion of slice 972/1000 to tmp array index 6
Page 32585/33000; a portion of slice 972/1000 to tmp array index 7
Page 32586/33000; a portion of slice 972/1000 to tmp array index 8
Page 32587/33000; a portion of slice 972/1000 to tmp array index 9
Page 32588/33000; a portion of slice 972/1000 to tmp array index 10
Page 32611/33000; a portion of slice 973/1000 to tmp array index 0
Page 32612/33000; a portion of slice 973/1000 to tmp array index 1
Page 32613/33000; a portion of slice 973/1000 to tmp array i

Page 32948/33000; a portion of slice 983/1000 to tmp array index 7
Page 32949/33000; a portion of slice 983/1000 to tmp array index 8
Page 32950/33000; a portion of slice 983/1000 to tmp array index 9
Page 32951/33000; a portion of slice 983/1000 to tmp array index 10
Page 32974/33000; a portion of slice 984/1000 to tmp array index 0
Page 32975/33000; a portion of slice 984/1000 to tmp array index 1
Page 32976/33000; a portion of slice 984/1000 to tmp array index 2
Page 32977/33000; a portion of slice 984/1000 to tmp array index 3
Page 32978/33000; a portion of slice 984/1000 to tmp array index 4
Page 32979/33000; a portion of slice 984/1000 to tmp array index 5
Page 32980/33000; a portion of slice 984/1000 to tmp array index 6
Page 32981/33000; a portion of slice 984/1000 to tmp array index 7
Page 32982/33000; a portion of slice 984/1000 to tmp array index 8
Page 32983/33000; a portion of slice 984/1000 to tmp array index 9
Page 32984/33000; a portion of slice 984/1000 to tmp array in

Page 33345/33000; a portion of slice 995/1000 to tmp array index 8
Page 33346/33000; a portion of slice 995/1000 to tmp array index 9
Page 33347/33000; a portion of slice 995/1000 to tmp array index 10
Page 33370/33000; a portion of slice 996/1000 to tmp array index 0
Page 33371/33000; a portion of slice 996/1000 to tmp array index 1
Page 33372/33000; a portion of slice 996/1000 to tmp array index 2
Page 33373/33000; a portion of slice 996/1000 to tmp array index 3
Page 33374/33000; a portion of slice 996/1000 to tmp array index 4
Page 33375/33000; a portion of slice 996/1000 to tmp array index 5
Page 33376/33000; a portion of slice 996/1000 to tmp array index 6
Page 33377/33000; a portion of slice 996/1000 to tmp array index 7
Page 33378/33000; a portion of slice 996/1000 to tmp array index 8
Page 33379/33000; a portion of slice 996/1000 to tmp array index 9
Page 33380/33000; a portion of slice 996/1000 to tmp array index 10
Page 33403/33000; a portion of slice 997/1000 to tmp array i

DLCForWBFMConfig(task_name='feature_alignment', experimenter='Charlie', datafiles=DLCForWBFMDatafiles(red_bigtiff_fname='D:\\More-stabilized-wbfm\\test2020-10-22_16-15-20_test4-channel-0-pco_camera1\\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf', green_bigtiff_fname='D:\\More-stabilized-wbfm\\test2020-10-22_16-15-20_test4-channel-1-pco_camera2\\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf', red_avi_fname='C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-06\\redfr1000_sl7_17.avi', green_avi_fname='C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-06\\greenfr1000_sl7_17.avi'), preprocessing=DLCForWBFMPreprocessing(start_volume=15, num_frames=1000, fps=5.0, alpha=0.15, num_total_slices=33, num_crop_slices=11, center_slice=12, red_and_green_mirrored=True), tracki

In [18]:
# Save the annotations in the config file
project_folder = config.get_dirname()

annotation_fname = 'CollectedData_feature_tracker.h5'
annotation_fname = os.path.join(project_folder, annotation_fname)

vid_fname = r'redfr1000_sl7_17.avi'
vid_fname = os.path.join(project_folder, vid_fname)

tracking = DLCForWBFMTracking(project_folder, vid_fname, annotation_fname)

config.tracking = tracking
save_config(config)

## Finally, label the video using the config file

In [19]:
import deeplabcut
from DLC_for_WBFM.utils.preprocessing.DLC_utils import create_dlc_project_from_config
from DLC_for_WBFM.utils.preprocessing.convert_matlab_annotations_to_DLC import csv_annotations2config_names

In [20]:
# Fix a bug
# config.datafiles.red_avi_fname = vid_fname
# save_config(config)
# vid_fname

'C:\\Users\\charles.fieseler\\Documents\\Current_work\\DLC_for_WBFM\\example-feature-workflow\\feature_alignment\\feature_alignment-Charlie-2021-01-06\\redfr1000_sl7_17.avi'

In [22]:
dlc_config = create_dlc_project_from_config(config, '1')

Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-01-06\f1-C-2021-01-06\videos"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-01-06\f1-C-2021-01-06\labeled-data"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-01-06\f1-C-2021-01-06\training-datasets"
Created "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-01-06\f1-C-2021-01-06\dlc-models"
Copying the videos
C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\example-feature-workflow\feature_alignment\feature_alignment-Charlie-2021-01-06\f1-C-2021-01-06\videos\redfr1000_sl7_17.avi
Generated "C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\e

In [23]:
# Make sure the dlc config file has all the neuron names
csv_annotations = config.tracking.annotation_fname.replace('h5', 'csv')
csv_annotations2config_names(dlc_config, csv_annotations)

Adding body part annotations for 174 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written


In [92]:
# Use my wrapper function
from DLC_for_WBFM.utils.feature_detection.visualize_using_dlc import *

video_fname = config.datafiles.red_avi_fname

scorer = 'feature_tracker'
df = pd.read_hdf(os.path.join(project_folder,"CollectedData_" + scorer + ".h5"), key="df_with_missing")


In [93]:
make_labeled_video_custom_annotations(dlc_config, video_fname, df)

  1%|▋                                                       | 13/1000 [00:00<00:08, 115.87it/s]

Duration of video [s]: 100.0, recorded with 10.0 fps!
Overall # of frames: 1000 with cropped frame dimensions: 900 700
Generating frames and creating video.


100%|██████████████████████████████████████████████████████| 1000/1000 [00:08<00:00, 120.66it/s]
